In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [24]:
import os
import csv
import shutil

def load_celeba_dataset(dataset_dir, output_dir, max_images=500):
    dataset = {'men_without_glasses': [], 'men_with_glasses': [], 'women_with_glasses': []}
    counts = {'men_without_glasses': 0, 'men_with_glasses': 0, 'women_with_glasses': 0}

    # Read attributes from CSV file
    attr_file_path = os.path.join(dataset_dir, 'list_attr_celeba.csv')
    with open(attr_file_path, 'r') as f:
        reader = csv.reader(f)
        attributes = next(reader)[1:]  # Skip first column (filename)

        for row in reader:
            if counts['men_without_glasses'] >= max_images and counts['men_with_glasses'] >= max_images and counts['women_with_glasses'] >= max_images:
                break  # Stop loading more images once we reach the limit for all categories

            filename = row[0]
            attr_values = row[1:]
            img_path = os.path.join(dataset_dir, 'img_align_celeba/img_align_celeba', filename)

            # Determine categories based on attributes
            if int(attr_values[attributes.index('Male')]) == 1:
                if int(attr_values[attributes.index('Eyeglasses')]) == 1 and counts['men_with_glasses'] < max_images:
                    dataset['men_with_glasses'].append(img_path)
                    counts['men_with_glasses'] += 1
                elif int(attr_values[attributes.index('Eyeglasses')]) == -1 and counts['men_without_glasses'] < max_images:
                    dataset['men_without_glasses'].append(img_path)
                    counts['men_without_glasses'] += 1
            else:
                if int(attr_values[attributes.index('Eyeglasses')]) == 1 and counts['women_with_glasses'] < max_images:
                    dataset['women_with_glasses'].append(img_path)
                    counts['women_with_glasses'] += 1

    # Split images into train and test sets
    split_ratio = 0.8  # Adjust this ratio as needed
    train = {'men_with_glasses': [], 'men_without_glasses': [], 'women_with_glasses': []}
    test = {'men_with_glasses': [], 'men_without_glasses': [], 'women_with_glasses': []}

    for category in ['men_with_glasses', 'men_without_glasses', 'women_with_glasses']:
        train_size = int(len(dataset[category]) * split_ratio)
        train[category] = dataset[category][:train_size]
        test[category] = dataset[category][train_size:]

    # Move images to respective directories
    for category in ['men_with_glasses', 'men_without_glasses', 'women_with_glasses']:
        for img_path in train[category]:
            shutil.copy(img_path, os.path.join(output_dir, category, 'train', os.path.basename(img_path)))
        for img_path in test[category]:
            shutil.copy(img_path, os.path.join(output_dir, category, 'test', os.path.basename(img_path)))

    return dataset

# Example usage
dataset_dir = '/kaggle/input/celeba-dataset/'
output_dir = '/kaggle/working/'  # Adjust this to your desired output directory
os.makedirs(os.path.join(output_dir, 'men_with_glasses', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'men_with_glasses', 'test'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'men_without_glasses', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'men_without_glasses', 'test'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'women_with_glasses', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'women_with_glasses', 'test'), exist_ok=True)
celeba_dataset = load_celeba_dataset(dataset_dir, output_dir, max_images=500)


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import scipy.misc as misc
from PIL import Image
import os

2024-04-30 11:57:53.826115: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 11:57:53.826210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 11:57:54.070795: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
epsilon = 1e-8
img_W = 128
img_H = 128
img_nums = 8000
batchsize = 5


In [4]:
# Convolutional Layer
def conv(inputs, nums_out, ksize, stride, padding, is_dis=False):
    c = int(inputs.shape[-1])
    W = tf.get_variable("W", shape=[ksize, ksize, c, nums_out], initializer=tf.random_normal_initializer(stddev=0.02))
    b = tf.get_variable("b", shape=[nums_out], initializer=tf.constant_initializer([0]))
    if is_dis:
        return tf.nn.conv2d(inputs, spectral_norm("SN",W), [1, stride, stride, 1], padding) + b
    else:
        return tf.nn.conv2d(inputs, W, [1, stride, stride, 1], padding) + b
# Transposed Convolutional Layer
def deconv(inputs, nums_out, ksize, stride):
    c = int(inputs.shape[-1])
    batch = int(inputs.shape[0])
    height = int(inputs.shape[1])
    width = int(inputs.shape[2])
    W = tf.get_variable("W", shape=[ksize, ksize, nums_out, c], initializer=tf.random_normal_initializer(stddev=0.02))
    b = tf.get_variable("b", shape=[nums_out], initializer=tf.constant_initializer([0.]))
    return tf.nn.conv2d_transpose(inputs, W, output_shape=[batch, height*stride, width*stride, nums_out], strides=[1, stride, stride, 1]) + b

# Instance Normalization
# def InstanceNorm(inputs):
#     mean, var = tf.nn.moments(inputs, axes=[1, 2], keep_dims=True)
#     scale = tf.get_variable("scale", shape=mean.shape[-1], initializer=tf.constant_initializer([1.]))
#     shift = tf.get_variable("shift", shape=mean.shape[-1], initializer=tf.constant_initializer([0.]))
#     return (inputs - mean) * scale / tf.sqrt(var + epsilon) + shift
def InstanceNorm(inputs, name=None):
    with tf.compat.v1.variable_scope(name, default_name='InstanceNorm'):
        mean, var = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        scale = tf.compat.v1.get_variable("scale", shape=mean.shape[-1], initializer=tf.constant_initializer([1.]), trainable=True)
        shift = tf.compat.v1.get_variable("shift", shape=mean.shape[-1], initializer=tf.constant_initializer([0.]), trainable=True)
        epsilon = 1e-8
        return (inputs - mean) * scale / tf.sqrt(var + epsilon) + shift


# Leaky ReLU activation
def leaky_relu(inputs, slope=0.2):
    return tf.maximum(slope*inputs, inputs)

# Spectral Normalization
def spectral_norm(name, w, iteration=1):
    # Implementation of Spectral Normalization
    # Code adapted from: https://github.com/taki0112/Spectral_Normalization-Tensorflow
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])
    with tf.variable_scope(name, reuse=False):
        u = tf.get_variable("u", [1, w_shape[-1]], initializer=tf.truncated_normal_initializer(), trainable=False)
    u_hat = u
    v_hat = None

    def l2_norm(v, eps=1e-12):
        return v / (tf.reduce_sum(v ** 2) ** 0.5 + eps)

    for i in range(iteration):
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = l2_norm(v_)
        u_ = tf.matmul(v_hat, w)
        u_hat = l2_norm(u_)
    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))
    w_norm = w / sigma
    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = tf.reshape(w_norm, w_shape)
    return w_norm

In [5]:
# # Discriminator Class
# class discriminator:
#     def __init__(self, name):
#         self.name = name

#     def __call__(self, inputs, reuse=False):
#         # Patch discriminator
#         inputs = tf.random_crop(inputs, [batchsize, 70, 70, 3])
#         with tf.variable_scope(self.name, reuse=reuse):
#             with tf.variable_scope("c64"):
#                 inputs = leaky_relu(conv(inputs, 64, 5, 2, "SAME", True))
#             with tf.variable_scope("c128"):
#                 inputs = leaky_relu(InstanceNorm(conv(inputs, 128, 5, 2, "SAME", True)))
#             with tf.variable_scope("c256"):
#                 inputs = leaky_relu(InstanceNorm(conv(inputs, 256, 5, 2, "SAME", True)))
#             with tf.variable_scope("c512"):
#                 inputs = leaky_relu(InstanceNorm(conv(inputs, 512, 5, 2, "SAME", True)))
#             with tf.variable_scope("fully_conv"):
#                 ksize = np.size(inputs, 1)
#                 inputs = tf.squeeze(conv(inputs, 1, ksize, 1, "VALID", True), axis=[1, 2, 3])
#         return inputs

#     @property
#     def var(self):
#         return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
class discriminator:
    def __init__(self, name):
        self.name = name

    def __call__(self, inputs, reuse=tf.AUTO_REUSE):
        with tf.compat.v1.variable_scope(self.name, reuse=reuse):
            # Patch discriminator
            inputs = tf.image.random_crop(inputs, [batchsize, 70, 70, 3])
            with tf.compat.v1.variable_scope("c64"):
                inputs = leaky_relu(conv(inputs, 64, 5, 2, "SAME", True))
            with tf.compat.v1.variable_scope("c128"):
                inputs = leaky_relu(InstanceNorm(conv(inputs, 128, 5, 2, "SAME", True)))
            with tf.compat.v1.variable_scope("c256"):
                inputs = leaky_relu(InstanceNorm(conv(inputs, 256, 5, 2, "SAME", True)))
            with tf.compat.v1.variable_scope("c512"):
                inputs = leaky_relu(InstanceNorm(conv(inputs, 512, 5, 2, "SAME", True)))
            with tf.compat.v1.variable_scope("fully_conv"):
                ksize = np.size(inputs, 1)
                inputs = tf.squeeze(conv(inputs, 1, ksize, 1, "VALID", True), axis=[1, 2, 3])
        return inputs

    @property
    def var(self):
        return tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope=self.name)


In [6]:
# # Generator Class
# class generator:
#     def __init__(self, name):
#         self.name = name

#     def __call__(self, inputs, reuse=False):
#         with tf.variable_scope(name_or_scope=self.name, reuse=reuse):
#             inputs = tf.pad(inputs, tf.constant([[0, 0], [3, 3], [3, 3], [0, 0]]))
#             with tf.variable_scope("c7s1-32"):
#                 inputs = tf.nn.relu(InstanceNorm(conv(inputs, 32, 7, 1, "VALID")))
#             with tf.variable_scope("d64"):
#                 inputs = tf.nn.relu(InstanceNorm(conv(inputs, 64, 3, 2, "SAME")))
#             with tf.variable_scope("d128"):
#                 inputs = tf.nn.relu(InstanceNorm(conv(inputs, 128, 3, 2, "SAME")))
#             for i in range(6):
#                 with tf.variable_scope("R"+str(i)):
#                     temp = inputs
#                     with tf.variable_scope("R_conv1"):
#                         inputs = tf.pad(inputs, tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]]), "REFLECT")
#                         inputs = tf.nn.relu(InstanceNorm(conv(inputs, 128, 3, 1, "VALID")))
#                     with tf.variable_scope("R_conv2"):
#                         inputs = tf.pad(inputs, tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]]), "REFLECT")
#                         inputs = InstanceNorm(conv(inputs, 128, 3, 1, "VALID"))
#                     inputs = temp + inputs
#             with tf.variable_scope("u64"):
#                 inputs = tf.nn.relu(InstanceNorm(deconv(inputs, 64, 3, 2)))
#             with tf.variable_scope("u32"):
#                 inputs = tf.nn.relu(InstanceNorm(deconv(inputs, 32, 3, 2)))
#             with tf.variable_scope("c7s1-3"):
#                 inputs = tf.nn.tanh((deconv(inputs, 3, 7, 1)))
#             return (inputs + 1.) * 127.5

#     @property
#     def var(self):
#         return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
class generator:
    def __init__(self, name):
        self.name = name

    def __call__(self, inputs, reuse=tf.AUTO_REUSE):
        with tf.compat.v1.variable_scope(name_or_scope=self.name, reuse=reuse):
            inputs = tf.pad(inputs, tf.constant([[0, 0], [3, 3], [3, 3], [0, 0]]))
            with tf.compat.v1.variable_scope("c7s1-32"):
                inputs = tf.nn.relu(InstanceNorm(conv(inputs, 32, 7, 1, "VALID")))
                inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
            with tf.compat.v1.variable_scope("d64"):
                inputs = tf.nn.relu(InstanceNorm(conv(inputs, 64, 3, 2, "SAME")))
                inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
            with tf.compat.v1.variable_scope("d128"):
                inputs = tf.nn.relu(InstanceNorm(conv(inputs, 128, 3, 2, "SAME")))
                inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
            for i in range(6):
                with tf.compat.v1.variable_scope("R" + str(i)):
                    temp = inputs
                    with tf.compat.v1.variable_scope("R_conv1"):
                        inputs = tf.pad(inputs, tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]]), "REFLECT")
                        inputs = tf.nn.relu(InstanceNorm(conv(inputs, 128, 3, 1, "VALID")))
                        inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
                    with tf.compat.v1.variable_scope("R_conv2"):
                        inputs = tf.pad(inputs, tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]]), "REFLECT")
                        inputs = InstanceNorm(conv(inputs, 128, 3, 1, "VALID"))
                        inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
                    inputs = temp + inputs
            with tf.compat.v1.variable_scope("u64"):
                inputs = tf.nn.relu(InstanceNorm(deconv(inputs, 64, 3, 2)))
                inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
            with tf.compat.v1.variable_scope("u32"):
                inputs = tf.nn.relu(InstanceNorm(deconv(inputs, 32, 3, 2)))
                inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
            with tf.compat.v1.variable_scope("c7s1-3"):
                inputs = tf.nn.tanh((deconv(inputs, 3, 7, 1)))
                inputs = tf.image.resize(inputs, [128, 128])  # Resize to match the input shape
            return (inputs + 1.) * 127.5

    @property
    def var(self):
        return tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope=self.name)


In [7]:
def resize_image(img_path):
    img = Image.open(img_path)
    img = img.resize((img_W, img_H), Image.BILINEAR)
    return np.array(img)

In [21]:
output_dir = '/kaggle/working/'
os.makedirs(os.path.join(output_dir, 'fake_Y'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'fake_X'), exist_ok=True)

In [38]:
def count_images_in_directory(directory):
    count = 0
    for file in os.listdir(directory):
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
            count += 1
    return count

# Example usage
directory_path = '/kaggle/working/women_with_glasses/train'
image_count = count_images_in_directory(directory_path)
print("Number of images:", image_count)

Number of images: 400


In [40]:
# CycleGAN Class
class CycleGAN:
    def __init__(self):
        self.X = tf.placeholder("float", shape=[batchsize, img_H, img_W, 3])
        self.Y = tf.placeholder("float", shape=[batchsize, img_H, img_W, 3])
        G = generator("G")
        F = generator("F")
        self.Dx = discriminator("Dx")
        self.Dy = discriminator("Dy")
        self.fake_X = F(self.Y)
        self.fake_Y = G(self.X)
        self.logits_real_X = self.Dx(self.X)
        self.logits_real_Y = self.Dy(self.Y)
        self.logits_fake_X = self.Dx(self.fake_X, True)
        self.logits_fake_Y = self.Dy(self.fake_Y, True)
        self.L_cyc = tf.reduce_mean(tf.abs(F(self.fake_Y, True) - self.X)) + tf.reduce_mean(tf.abs(G(self.fake_X, True) - self.Y))
        self.d_loss_Y = -tf.reduce_mean(self.logits_real_Y) + tf.reduce_mean(self.logits_fake_Y)
        self.d_loss_X = -tf.reduce_mean(self.logits_real_X) + tf.reduce_mean(self.logits_fake_X)
        self.g_loss_Y = -tf.reduce_mean(self.logits_fake_Y) + 10. * self.L_cyc
        self.g_loss_X = -tf.reduce_mean(self.logits_fake_X) + 10. * self.L_cyc
#         self.Dx_opt = tf.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.d_loss_X, var_list=[self.Dx.var])
#         self.Dy_opt = tf.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.d_loss_Y, var_list=[self.Dy.var])
#         self.G_opt = tf.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.g_loss_Y, var_list=G.var)
#         self.F_opt = tf.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.g_loss_X, var_list=F.var)
        with tf.compat.v1.variable_scope("optimizer", reuse=tf.compat.v1.AUTO_REUSE):
            self.Dx_opt = tf.compat.v1.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.d_loss_X, var_list=[self.Dx.var])
            self.Dy_opt = tf.compat.v1.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.d_loss_Y, var_list=[self.Dy.var])
            self.G_opt = tf.compat.v1.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.g_loss_Y, var_list=G.var)
            self.F_opt = tf.compat.v1.train.AdamOptimizer(2e-4, beta1=0., beta2=0.9).minimize(self.g_loss_X, var_list=F.var)



        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.train()

    def train(self):
        Y_path = "/kaggle/working/women_with_glasses/train/"
        X_path = "/kaggle/working/men_with_glasses/train/"
        Y = os.listdir(Y_path)[:img_nums]
        X = os.listdir(X_path)[:img_nums]
        nums = len(Y)
        saver = tf.train.Saver()
        for epoch in range(10):
            print(epoch);
            for i in range(int(nums / batchsize) - 1):
                X_img = np.zeros([batchsize, img_H, img_W, 3])
                Y_img = np.zeros([batchsize, img_H, img_W, 3])
                for j in np.arange(i * batchsize, i * batchsize + batchsize, 1):
                    img = resize_image(X_path + X[j])
                    X_img[j - i * batchsize, :, :, :] = img
                    img = resize_image(Y_path + Y[j])
                    Y_img[j - i * batchsize, :, :, :] = img
                self.sess.run(self.Dy_opt, feed_dict={self.X: X_img, self.Y: Y_img})
                self.sess.run(self.Dx_opt, feed_dict={self.X: X_img, self.Y: Y_img})
                self.sess.run(self.G_opt, feed_dict={self.X: X_img, self.Y: Y_img})
                self.sess.run(self.F_opt, feed_dict={self.X: X_img, self.Y: Y_img})
                if i % 10 == 0:
                    [d_loss_X, d_loss_Y, g_loss_Y, g_loss_X, fake_X, fake_Y, cyc_loss] = \
                        self.sess.run([self.d_loss_X, self.d_loss_Y, self.g_loss_Y, self.g_loss_X, self.fake_X, self.fake_Y, self.L_cyc], feed_dict={self.X: X_img, self.Y: Y_img})
                    print("epoch: %d, step: %d, d_loss_X: %g, d_loss_Y: %g, g_loss_X: %g, g_loss_Y: %g, cyc_loss: %g"%(epoch, i, d_loss_X, d_loss_Y, g_loss_X, g_loss_Y, cyc_loss))
                    Image.fromarray(np.uint8(fake_Y)[0, :, :, :]).save("/kaggle/working/fake_Y/"+str(epoch)+"_"+str(i)+".jpg")
                    Image.fromarray(np.uint8(fake_X)[0, :, :, :]).save("/kaggle/working/fake_X/" + str(epoch) + "_" + str(i) + ".jpg")
            saver.save(self.sess, "./save_para//CycleGAN_man_woman.ckpt")
\
if __name__ == "__main__":
    cyc = CycleGAN()

0
epoch: 0, step: 0, d_loss_X: -0.662287, d_loss_Y: -0.585405, g_loss_X: 1266.29, g_loss_Y: 1264.96, cyc_loss: 126.562
epoch: 0, step: 10, d_loss_X: 1.36976, d_loss_Y: 2.26612, g_loss_X: 612.414, g_loss_Y: 613.968, cyc_loss: 61.7772
epoch: 0, step: 20, d_loss_X: -7.28175, d_loss_Y: -4.82588, g_loss_X: 778.058, g_loss_Y: 775.683, cyc_loss: 78.0544
epoch: 0, step: 30, d_loss_X: -5.50047, d_loss_Y: -2.75253, g_loss_X: 631.894, g_loss_Y: 636.349, cyc_loss: 64.0374
epoch: 0, step: 40, d_loss_X: 11.7083, d_loss_Y: 0.358695, g_loss_X: 694.663, g_loss_Y: 721.594, cyc_loss: 72.3764
epoch: 0, step: 50, d_loss_X: -3.22335, d_loss_Y: -22.5157, g_loss_X: 1102.53, g_loss_Y: 1124.25, cyc_loss: 111.807
epoch: 0, step: 60, d_loss_X: -7.94127, d_loss_Y: -5.23399, g_loss_X: 1519.51, g_loss_Y: 1502.39, cyc_loss: 151.623
epoch: 0, step: 70, d_loss_X: -29.6034, d_loss_Y: -0.147729, g_loss_X: 756.034, g_loss_Y: 741.365, cyc_loss: 74.7483
1
epoch: 1, step: 0, d_loss_X: -2.2395, d_loss_Y: -6.08209, g_loss_X: 6

In [46]:
input_image_path = "/kaggle/working/men_with_glasses/test/008007.jpg"

# Load the input image and resize it
input_image = resize_image(input_image_path)

# Generate the image with glasses
generated_image = cyc.sess.run(cyc.fake_X, feed_dict={cyc.Y: np.tile([input_image], (5, 1, 1, 1))})


# Convert the generated image to uint8 and save it
generated_image = np.uint8(generated_image[0])
output_image = Image.fromarray(generated_image)
output_image.save("/kaggle/working/generated_image_women_with_glasses.jpg")

In [ ]:
input_image_path = "/kaggle/working/men_with_glasses/test/001721.jpg"

# Load the input image and resize it
input_image = resize_image(input_image_path)

# Generate the image with glasses
generated_image = cyc.sess.run(cyc.fake_X, feed_dict={cyc.Y: np.tile([input_image], (5, 1, 1, 1))})


# Convert the generated image to uint8 and save it
generated_image = np.uint8(generated_image[0])
output_image = Image.fromarray(generated_image)
output_image.save("/kaggle/working/generated_image_without_glasses2.jpg")

In [53]:
input_image_path = "/kaggle/working/women_with_glasses/test/035030.jpg"

# Load the input image and resize it
input_image = resize_image(input_image_path)

# Generate the image with glasses
generated_image = cyc.sess.run(cyc.fake_Y, feed_dict={cyc.X: np.tile([input_image], (5, 1, 1, 1))})


# Convert the generated image to uint8 and save it
generated_image = np.uint8(generated_image[0])
output_image = Image.fromarray(generated_image)
output_image.save("/kaggle/working/generated_image_men_with_glasses.jpg")

In [51]:
input_image_path = "/kaggle/working/women_with_glasses/train/007468.jpg"

# Load the input image and resize it
input_image = resize_image(input_image_path)

# Generate the image with glasses
generated_image = cyc.sess.run(cyc.fake_Y, feed_dict={cyc.X: np.tile([input_image], (5, 1, 1, 1))})


# Convert the generated image to uint8 and save it
generated_image = np.uint8(generated_image[0])
output_image = Image.fromarray(generated_image)
output_image.save("/kaggle/working/generated_image_with_glasses2.jpg")